<h1> Install pyspark library </h1>

In [1]:
!pip install pyspark

In [2]:
#import pyspark library 
import pyspark

In [3]:
#import spark session library
from pyspark.sql import SparkSession

In [4]:
#Create SparkSession object
spark = SparkSession.builder \
                .master('local[*]') \
                .appName('random_forest_regression') \
                .getOrCreate()

<h1> Create DataFrame </h1>

In [5]:
#Tocreate dataframe form external datasets
df = spark.read.option("header","true").csv(r"C:\Users\User\Downloads\partitions\Data\*")

In [6]:
df.show()

+----+-------+-----+----------+---------+----------+-------------+---------+-------+-------+---------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+---------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+-------------+-----------+-----------+-------------+--------------

<h1> Data Preparation </h1>

In [7]:
#create new dataframe as per required columns for prediction 
AirlineDF = df.select("Origin","Dest","AirTime","Distance")

In [8]:
#cache data in-memory for fast read write operation 
AirlineDF.cache()

DataFrame[Origin: string, Dest: string, AirTime: string, Distance: string]

In [9]:
#check data in new dataframe 
AirlineDF.show()

+------+----+-------+--------+
|Origin|Dest|AirTime|Distance|
+------+----+-------+--------+
|   JFK| LAX| 338.00| 2475.00|
|   JFK| LAX| 349.00| 2475.00|
|   JFK| LAX| 370.00| 2475.00|
|   JFK| LAX| 350.00| 2475.00|
|   JFK| LAX| 335.00| 2475.00|
|   JFK| LAX| 336.00| 2475.00|
|   JFK| LAX| 380.00| 2475.00|
|   JFK| LAX| 359.00| 2475.00|
|   JFK| LAX| 368.00| 2475.00|
|   JFK| LAX| 356.00| 2475.00|
|   JFK| LAX| 353.00| 2475.00|
|   JFK| LAX| 332.00| 2475.00|
|   JFK| LAX| 339.00| 2475.00|
|   JFK| LAX| 339.00| 2475.00|
|   JFK| LAX| 335.00| 2475.00|
|   JFK| LAX| 340.00| 2475.00|
|   JFK| LAX| 327.00| 2475.00|
|   JFK| LAX| 308.00| 2475.00|
|   JFK| LAX| 315.00| 2475.00|
|   JFK| LAX| 323.00| 2475.00|
+------+----+-------+--------+
only showing top 20 rows



In [10]:
#check data types of each columns
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)



<h1> We need to change data types of columns </h1>

In [11]:
from pyspark.sql.types import IntegerType

In [12]:
AirlineDF = AirlineDF.withColumn('Distance', AirlineDF["Distance"].cast(IntegerType()))

In [13]:
AirlineDF = AirlineDF.withColumn('AirTime', AirlineDF["AirTime"].cast(IntegerType()))

In [14]:
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: integer (nullable = true)



<h1> Check is there any null values in DataFrame </h1>

In [16]:
#import library for sql function col
from pyspark.sql.functions import col

In [17]:
#check null values in each columns
print(AirlineDF.where(col("Origin").isNull()).count())
print(AirlineDF.where(col("Dest").isNull()).count())
print(AirlineDF.where(col("AirTime").isNull()).count())
print(AirlineDF.where(col("Distance").isNull()).count())

0
0
0
0


<h1> we need convert distance miles into km </h1>

In [20]:
from pyspark.sql.functions import round

#Convert 'mile' to 'km'
AirlineDF = AirlineDF.withColumn('Distance', round(AirlineDF.Distance * 1.60934, 0))
AirlineDF.show()

+------+----+-------+--------+
|Origin|Dest|AirTime|Distance|
+------+----+-------+--------+
|   JFK| LAX|    338|  3983.0|
|   JFK| LAX|    349|  3983.0|
|   JFK| LAX|    370|  3983.0|
|   JFK| LAX|    350|  3983.0|
|   JFK| LAX|    335|  3983.0|
|   JFK| LAX|    336|  3983.0|
|   JFK| LAX|    380|  3983.0|
|   JFK| LAX|    359|  3983.0|
|   JFK| LAX|    368|  3983.0|
|   JFK| LAX|    356|  3983.0|
|   JFK| LAX|    353|  3983.0|
|   JFK| LAX|    332|  3983.0|
|   JFK| LAX|    339|  3983.0|
|   JFK| LAX|    339|  3983.0|
|   JFK| LAX|    335|  3983.0|
|   JFK| LAX|    340|  3983.0|
|   JFK| LAX|    327|  3983.0|
|   JFK| LAX|    308|  3983.0|
|   JFK| LAX|    315|  3983.0|
|   JFK| LAX|    323|  3983.0|
+------+----+-------+--------+
only showing top 20 rows



In [21]:
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: double (nullable = true)



<h1>Vectorize the features </h1>

In [18]:
from pyspark.ml.feature import *

In [19]:
from pyspark.ml.feature import VectorAssembler

In [22]:
vectorizer = VectorAssembler()
vectorizer.setInputCols(["Distance"])
vectorizer.setOutputCol("features")

df_vect = vectorizer.transform(AirlineDF)


In [23]:
df_vect.show()

+------+----+-------+--------+--------+
|Origin|Dest|AirTime|Distance|features|
+------+----+-------+--------+--------+
|   JFK| LAX|    338|  3983.0|[3983.0]|
|   JFK| LAX|    349|  3983.0|[3983.0]|
|   JFK| LAX|    370|  3983.0|[3983.0]|
|   JFK| LAX|    350|  3983.0|[3983.0]|
|   JFK| LAX|    335|  3983.0|[3983.0]|
|   JFK| LAX|    336|  3983.0|[3983.0]|
|   JFK| LAX|    380|  3983.0|[3983.0]|
|   JFK| LAX|    359|  3983.0|[3983.0]|
|   JFK| LAX|    368|  3983.0|[3983.0]|
|   JFK| LAX|    356|  3983.0|[3983.0]|
|   JFK| LAX|    353|  3983.0|[3983.0]|
|   JFK| LAX|    332|  3983.0|[3983.0]|
|   JFK| LAX|    339|  3983.0|[3983.0]|
|   JFK| LAX|    339|  3983.0|[3983.0]|
|   JFK| LAX|    335|  3983.0|[3983.0]|
|   JFK| LAX|    340|  3983.0|[3983.0]|
|   JFK| LAX|    327|  3983.0|[3983.0]|
|   JFK| LAX|    308|  3983.0|[3983.0]|
|   JFK| LAX|    315|  3983.0|[3983.0]|
|   JFK| LAX|    323|  3983.0|[3983.0]|
+------+----+-------+--------+--------+
only showing top 20 rows



In [24]:
print(vectorizer.explainParams())

handleInvalid: How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip'). (default: error)
inputCols: input column names. (current: ['Distance'])
outputCol: output column name. (default: VectorAssembler_ec76019d59e1__output, current: features)


<h1> Train test data Splitting </h1>

In [25]:
flights_train, flights_test = df_vect.randomSplit([0.8, 0.2])

In [26]:
flights_train.show()

+------+----+-------+--------+--------+
|Origin|Dest|AirTime|Distance|features|
+------+----+-------+--------+--------+
|   ABE| CLT|     70|   774.0| [774.0]|
|   ABE| CLT|     72|   774.0| [774.0]|
|   ABE| CLT|     74|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     77|   774.0| [774.0]|
|   ABE| CLT|     78|   774.0| [774.0]|
|   ABE| CLT|     79|   774.0| [774.0]|
|   ABE| CLT|     79|   774.0| [774.0]|
|   ABE| CLT|     82|   774.0| [774.0]|
|   ABE| CLT|     82|   774.0| [774.0]|
|   ABE| CLT|     83|   774.0| [774.0]|
|   ABE| CLT|     83|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     84|   774.0| [774.0]|
|   ABE| CLT|     86|   774.0| [774.0]|
|   ABE| CLT|     87|   774.0| [774.0]|
+------+----+-------+--------+--------+
only showing top 20 rows



<h1> RandomForest Regression Model Training </h1>

In [27]:
from pyspark.ml.regression import RandomForestRegressor 

In [28]:
#Train RandomForest model/
rf = RandomForestRegressor(featuresCol = "features")
print(rf.explainParams())

bootstrap: Whether bootstrap samples are used when building trees. (default: True)
cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the featur

In [30]:
rf.setLabelCol("AirTime")
rf.setFeaturesCol("features")
model = rf.fit(flights_train)

In [31]:
type(model)

pyspark.ml.regression.RandomForestRegressionModel

<h1> Model Testing </h1>

In [33]:
df_pred = model.transform(flights_test)
df_pred.show()

+------+----+-------+--------+--------+------------------+
|Origin|Dest|AirTime|Distance|features|        prediction|
+------+----+-------+--------+--------+------------------+
|   ABE| CLT|     76|   774.0| [774.0]| 75.62403710690569|
|   ABE| CLT|     87|   774.0| [774.0]| 75.62403710690569|
|   ABE| CLT|     87|   774.0| [774.0]| 75.62403710690569|
|   ABE| CLT|     90|   774.0| [774.0]| 75.62403710690569|
|   ABE| CLT|     94|   774.0| [774.0]| 75.62403710690569|
|   ABE| CLT|    101|   774.0| [774.0]| 75.62403710690569|
|   ABE| CLT|    128|   774.0| [774.0]| 75.62403710690569|
|   ABE| PHL|     19|    89.0|  [89.0]|48.930012408988766|
|   ABE| PHL|     19|    89.0|  [89.0]|48.930012408988766|
|   ABE| PHL|     21|    89.0|  [89.0]|48.930012408988766|
|   ABE| PHL|     23|    89.0|  [89.0]|48.930012408988766|
|   ABE| PHL|     24|    89.0|  [89.0]|48.930012408988766|
|   ABE| PHL|     24|    89.0|  [89.0]|48.930012408988766|
|   ABE| PHL|     25|    89.0|  [89.0]|48.93001240898876

<h1> View model summary </h1>

In [34]:
from pyspark.ml.evaluation import RegressionEvaluator

In [36]:
evaluator = RegressionEvaluator(labelCol = "AirTime", predictionCol="prediction", metricName = "rmse")
rmse = evaluator.evaluate(df_pred)
print("Root mean Squarred Error (RMSE) on test data = %g" % rmse)

Root mean Squarred Error (RMSE) on test data = 18.7878
